In [3]:
from source.regency import Regency
from source.DQNAgent import DQNAgent
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')


Game = Regency()
clear_output()
for a in range(1):
    Game.random_override = {}
    Game.random_override['JR'] = 'Unrest or Rebellion'
    Game.random_events(Threshold=99)
    Game.collect_regency_points()
    Game.domain_initiative()
    Game.collect_gold_bars()
    Game.maintenance_costs(True)
    print(Game.Seasons[Game.Season]['Season'])
    Game.take_domain_actions(None)
Game.clear_screen()
Game.Seasons[Game.Season]['Season']

,Regent,Player,Random Event,Regency Points,Initiative,Gold Bars,Revenue,Cost,Court
0,GG,False,No Event,77,21,42,32,30,Rich
1,TCV,False,No Event,1,20,0,1,1,Dormant
2,LPA,False,No Event,11,20,7,11,4,Bare
3,CSH,False,No Event,10,15,6,9,3,Bare
4,TF,False,No Event,2,15,1,2,1,Dormant
5,Sw,False,Corruption/Crime,11,14,0,0,0,Dormant
6,GH,False,Feud,10,14,4,7,3,Bare
7,Sw2,False,Diplomatic Matter,24,13,3,0,0,Dormant
8,DW,False,Brigandage,71,13,32,10,11,Rich
9,Ch,False,Diplomatic Matter,28,13,0,0,0,Dormant


In [6]:
Regent = 'Sw2'
self=Game
import pandas as pd
import numpy as np

temp = self.Holdings[self.Holdings['Regent']==Regent].copy()
temp = temp[temp['Type']=='Source']
temp['Other'] = temp['Provence']
temp = pd.merge(temp[['Provence', 'Type','Level']], temp[['Other', 'Type']], on='Type', how='outer')
temp = temp[temp['Provence'] != temp['Other']]
temp = pd.merge(temp, self.LeyLines, on=['Provence', 'Other'], how='left').fillna(0)
temp = temp[temp['Regent']==0]
temp['Roll'] = np.random.randint(1,10,temp.shape[0])+temp['Level']
temp=temp.sort_values('Roll', ascending=False)

temp

,Provence,Type,Level,Other,Regent,Roll
28,Sunken Lands,Source,6,Moergen,0,15
29,Sunken Lands,Source,6,Moriel,0,14
25,Sunken Lands,Source,6,Bogsend,0,13
27,Sunken Lands,Source,6,Gulfport,0,12
18,Moergen,Source,3,Moriel,0,12
5,Ghalliere,Source,3,Bogsend,0,11
8,Ghalliere,Source,3,Moriel,0,11
11,Gulfport,Source,2,Ghalliere,0,10
24,Moriel,Source,1,Sunken Lands,0,10
23,Moriel,Source,1,Moergen,0,10


In [40]:
import networkx as nx
Provence = 'Sunken Lands'
Target =  'Moriel'
Regent = 'Sw2'

G = nx.Graph()
G.add_nodes_from(list(self.Provences['Provence']))
Gl = G.copy()

G.add_edges_from([(row['Provence'], row['Neighbor']) for i, row in self.Geography.iterrows()])
Gl.add_edges_from([(row['Provence'], row['Other']) for i, row in self.LeyLines[self.LeyLines['Regent']==Regent].iterrows()])

provences = [Provence] + [a[1] for a in Gl.edges(Provence)]
targets = [Target] + [a[1] for a in Gl.edges(Target)]

shortest = 9000
pair = (Provence, Target)
for p in provences:
    for t in targets:
        a = nx.shortest_path_length(G, p, t, 'Border')
        if a < shortest:
            shortest = a
            pair = (p,t)
            
print(shortest, pair)

1 ('Moergen', 'Moriel')


In [36]:
self.LeyLines = self.LeyLines.append(pd.DataFrame([['Sw2', 'Moergen', 'Sunken Lands']], columns=self.LeyLines.keys()))

In [37]:
self.LeyLines

,Regent,Provence,Other
0,Moergen,Sunken Lands,Sw2
0,Moergen,Sunken Lands,Sw2
0,Sw2,Moergen,Sunken Lands


In [28]:
[a[1] for a in Gl.edges(Provence)]

[]

In [23]:
G.edges(Provence)

EdgeDataView([('Sunken Lands', 'Gulfport'), ('Sunken Lands', 'Moergen'), ('Sunken Lands', 'Bogsend'), ('Sunken Lands', 'Ruorven')])

In [21]:
Gl.edges('Moergen')

EdgeDataView([])

In [38]:
Gl.add_edges_from([(row['Provence'], row['Other']) for i, row in self.LeyLines[self.LeyLines['Regent']==Regent].iterrows()])

In [39]:
[(row['Provence'], row['Other']) for i, row in self.LeyLines[self.LeyLines['Regent']==Regent].iterrows()]

[('Moergen', 'Sunken Lands')]

In [35]:
elf.LeyLines[self.LeyLines['Regent']==Regent]

,Regent,Provence,Other
0,Moergen,Sunken Lands,Sw2
0,Moergen,Sunken Lands,Sw2


In [41]:
nx.shortest_path(G, pair[0], pair[1], 'Border')

['Moergen', 'Moriel']